<a href="https://colab.research.google.com/github/isacancabuk/toyota-model-labelling/blob/main/notebooks/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
%%bash
# 1) GitHub repo’sunu klonla
git clone https://github.com/isacancabuk/toyota-model-labelling.git
cd toyota-model-labelling

# 2) Python paketlerini yükle
pip install -r requirements.txt

fatal: destination path 'toyota-model-labelling' already exists and is not an empty directory.


In [10]:
# Tek bir import yeterli; Colab hücreler arası bu import'u unutmaz
import os

# Kaç görsel var, ilk 5 dosya ismi neler?
dataset = 'data/4runner'
print("Toplam görsel:", len(os.listdir(dataset)))
print("İlk 5:", os.listdir(dataset)[:5])

FileNotFoundError: [Errno 2] No such file or directory: 'data/4runner'

In [4]:
# data/4runner içindeki görselleri %80–20 oranında train/val klasörlerine ayır
import os, random, shutil
from pathlib import Path

src = Path('data/4runner')
train_dir = Path('data/4runner_split/train')
val_dir   = Path('data/4runner_split/val')

for d in (train_dir, val_dir):
    d.mkdir(parents=True, exist_ok=True)

imgs = list(src.glob('*.jpg'))
random.shuffle(imgs)
split = int(0.8 * len(imgs))

for i, img in enumerate(imgs):
    target = train_dir if i < split else val_dir
    shutil.copy(img, target / img.name)

print(f"Train: {len(list(train_dir.glob('*')))} | Val: {len(list(val_dir.glob('*')))}")

Train: 0 | Val: 0


In [41]:
%%bash
# Ultralytics YOLOv5 kodunu indir ve bağımlılıkları yükle
git clone https://github.com/ultralytics/yolov5.git /content/yolov5
cd /content/yolov5
pip install -q -r requirements.txt

Cloning into '/content/yolov5'...


In [3]:
import torch
from pathlib import Path

# Modeli yükle (COCO ön-eğitimli YOLOv5s)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

CAR_ID = 2
val_images = list(Path('data/4runner_split/val').glob('*.jpg'))
total = len(val_images)
detected = 0

for img in val_images:
    results = model(str(img), conf=0.25)
    # default conf threshold
    df = results.pandas().xyxy[0]
    if (df['class'] == CAR_ID).any():
        detected += 1

print(f"Detected in {detected}/{total} images → Recall: {detected/total:.2%}")

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-5-12 Python-3.11.12 torch-2.6.0+cu124 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


total= 0


ZeroDivisionError: division by zero

In [ ]:
%%bash
# W&B entegrasyonunu kapat
export WANDB_MODE=disabled

# YOLOv5 klasörüne geç
cd /content/yolov5

# Eğitim komutu
python train.py \
  --img 640 \
  --batch 8 \
  --epochs 50 \
  --data /content/toyota-model-labelling/data/4runner.yaml \
  --weights yolov5s.pt \
  --name 4runner_finetune \
  --cache